In [114]:
import pandas as pd
from jiwer import wer
import string

# Prepocessing

In [115]:
def sort_by_file_name(df):
    df[['part1', 'part2', 'part3']] = df['file_name'].str.split('-', expand=True)
    df['part1'] = df['part1'].astype(int)
    df['part2'] = df['part2'].astype(int)
    df['part3'] = df['part3'].astype(int)
    df = df.sort_values(by=['part1', 'part2', 'part3'])
    df = df.drop(columns=['part1', 'part2', 'part3'])
    df = df.reset_index(drop=True)
    return df

def preprocess_csv(csv_path, rm_stutter=False):
    df = pd.read_csv(csv_path, index_col=0)
    df['file_name'] = df['file_name'].str.replace('.flac', '', regex=False)
    df['file_name'] = df['file_name'].str.replace('.csv', '', regex=False)
    df.iloc[:, 1] = df.iloc[:, 1].apply(lambda x: x.translate(str.maketrans('', '', string.punctuation)).lstrip())
    if rm_stutter==True:
        df.iloc[:, 1] = df.iloc[:, 1].str.replace('stutter', '', case=False)
    df = sort_by_file_name(df)
    return df

In [116]:
df_librispeech_gt = preprocess_csv("librispeech_result/gt_librispeech.csv")
df_librispeech_gt.rename(columns={'ground_truth': 'librispeech_gt'}, inplace=True)

df_librispeech_whisper = preprocess_csv("librispeech_result/whisper_librispeech.csv")
df_librispeech_whisper.rename(columns={'prediction': 'librispeech_whisper'}, inplace=True)

df_librispeech_wav2vec = preprocess_csv("librispeech_result/wav2vec_librispeech.csv")
df_librispeech_wav2vec.rename(columns={'prediction': 'librispeech_wav2vec'}, inplace=True)

# df_librispeech_googlecloud = preprocess_csv("librispeech_result/googlecloud_librispeech.csv")
# df_librispeech_googlecloud.rename(columns={'prediction': 'librispeech_googlecloud'}, inplace=True)
# display(df_librispeech_googlecloud.head(2))


df_libristutter_gt = preprocess_csv("libristutter_result/gt_libristutter.csv", rm_stutter=True)
df_libristutter_gt.rename(columns={'gt_transcriptions': 'libristutter_gt'}, inplace=True)
display(df_libristutter_gt.head())

df_libristutter_whisper = preprocess_csv("libristutter_result/whisper_libristutter.csv")
df_libristutter_whisper.rename(columns={'prediction': 'libristutter_whisper'}, inplace=True)
display(df_libristutter_whisper.head())


df_libristutter_wav2vec = preprocess_csv("libristutter_result/wav2vec_libristutter.csv")
df_libristutter_wav2vec.rename(columns={'prediction': 'libristutter_wav2vec'}, inplace=True)

# df_libristutter_googlecloud = preprocess_csv("librispeech_result/googlecloud_librispeech.csv")
# df_libristutter_googlecloud.rename(columns={'prediction': 'librispeech_googlecloud'}, inplace=True)
# display(df_libristutter_googlecloud.head(2))

,file_name,libristutter_gt,total_stutter
0,78-368-0000,chapter 1 mrs rachel lynde is surprised mrs ...,11
1,78-368-0001,it had its source away back in the woods at th...,8
2,78-368-0002,for not even if brooke could run past mrs rach...,0
3,78-368-0003,and that if she noticed anything odd or out of...,6
4,78-368-0004,but mrs rachel lynde was one of those capable...,7


,file_name,libristutter_whisper
0,78-368-0000,chapter 23 it was eight oclock when we landed ...
1,78-368-0001,the wind which had fallen in the south now ros...
2,78-368-0002,the clouds swept across its swifter than the f...
3,78-368-0003,that were beginning to rise suddenly a heavy s...
4,78-368-0004,and not shrink from the conflict until my own ...


# Calculate WER

In [117]:
def calculate_wer(df_model, df_gt, output_filename):
    df_wer = pd.merge(df_model, df_gt, on='file_name')
    df_wer[output_filename] = df_wer.apply(lambda row: wer(row.iloc[1], row.iloc[2]), axis=1)
    df_wer.to_csv(f'wer_scores_csv/{output_filename}.csv', index=False)
    return df_wer

In [130]:
df_librispeech_whisper_wer = calculate_wer(df_librispeech_whisper, df_librispeech_gt, 'wer_librispeech_whisper')
df_librispeech_wav2vec_wer = calculate_wer(df_librispeech_wav2vec, df_librispeech_gt, 'wer_librispeech_wav2vec')
# df_librispeech_googlecloud_wer = calculate_wer(df_librispeech_googlecloud, df_librispeech_gt, 'wer_librispeech_googlecloud')

df_libristutter_whisper_wer = calculate_wer(df_libristutter_whisper, df_librispeech_gt, 'wer_libristutter_whisper')
df_libristutter_wav2vec_wer = calculate_wer(df_libristutter_wav2vec, df_librispeech_gt, 'wer_libristutter_wav2vec')
# df_libristutter_googlecloud_wer = calculate_wer(df_libristutter_googlecloud, df_librispeech_gt, 'wer_libristutter_googlecloud')

# Create the final csv file ‘wer_librispeech_libristutter.csv’

In [140]:
def merge_dfs(dfs):
    df_list = [df[['file_name', df.columns[-1]]] for df in dfs] # Extract file_name and WER from each df
    merged_df = df_list[0]
    for df in df_list[1:]:
        merged_df = merged_df.merge(df, on='file_name', how='outer')
    return merged_df

In [149]:
dfs = [
    df_librispeech_whisper_wer, df_librispeech_wav2vec_wer, # df_librispeech_googlecloud_wer,
    df_libristutter_whisper_wer, df_libristutter_wav2vec_wer, # df_libristutter_googlecloud_wer
]
df_librispeech_libristutter = merge_dfs(dfs)
df_librispeech_libristutter.to_csv('wer_scores_csv/wer_librispeech_libristutter.csv', index=False)
df_librispeech_libristutter.iloc[1310:1315]

,file_name,wer_librispeech_whisper,wer_librispeech_wav2vec,wer_libristutter_whisper,wer_libristutter_wav2vec
1310,1867-154071-0019,0.187500,0.0625,0.277778,0.166667
1311,1867-154071-0020,0.055556,0.0000,0.227273,0.270833
1312,1867-154071-0021,0.000000,0.0000,0.000000,0.047619
1313,1867-154071-0022,0.000000,0.0000,0.000000,0.000000
1314,1867-154071-0023,0.032258,0.0000,0.142857,0.138889


# Result Analysis

In [142]:
print('Mean WER:')
print('LibriSpeech Whisper:', df_librispeech_libristutter['wer_librispeech_whisper'].mean())
print('LibriSpeech Wav2vec:', df_librispeech_libristutter['wer_librispeech_wav2vec'].mean())
# print('LibriSpeech GoogleCloud:', df_librispeech_libristutter['wer_librispeech_googlecloud'].mean())
print('LibriStutter Whisper:', df_librispeech_libristutter['wer_libristutter_whisper'].mean())
print('LibriStutter Wav2vec:', df_librispeech_libristutter['wer_libristutter_wav2vec'].mean())
# print('LibriSpeech GoogleCloud:', df_librispeech_libristutter['wer_libristutter_googlecloud'].mean())

Mean WER:
LibriSpeech Whisper: 0.07270720071708915
LibriSpeech Wav2vec: 0.014986007919929566
LibriStutter Whisper: 0.12012099638035054
LibriStutter Wav2vec: 0.09635759641553532
